#### `confirm_wave_calcs` - Check my calcs with spotter data  
Based on code and advice from Isabel Houghton, Jake Davis, and Jim Thomson



In [1]:
import os
import pickle
from datetime import datetime, timezone
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib.colors
import numpy as np

import xarray as xr
import xroms
import matplotlib.pyplot as plt
from wavespectra import read_ww3, read_swan, read_ndbc, read_netcdf
from roguewave.wavespectra.estimators.estimate import estimate_directional_distribution

%run -i wave_stats.py


Exception: File `'wave_stats.py'` not found.

#### Define some routines

#### Start by loading spotter data 

In [ ]:
DATA_DIRECTORY = '/vortexfs1/home/csherwood/proj/NOPP/buoy_data/'
DATA_FILENAME = 'hurricane_ian_spotter_data_v1.pickle'
#MODEL_DIRECTORY = '/vortexfs1/scratch/jwarner/Ian2022/ian10/'
MODEL_DIRECTORY = '/vortexfs1/share/usgs-share/Projects/Ian2022/ian10/drifters/'

# This runs the file with the new ndbc buoy input
#%run -i /vortexfs1/home/csherwood/src/wavespectra/wavespectra/input/ndbc.py

with open(os.path.join(DATA_DIRECTORY, DATA_FILENAME), 'rb') as handle:
    spotter = pickle.load(handle)

# `spotter` is a python dictionary of Pandas DataFrames, keyed by
# each drifter ID. The drifter ids can then be accessed as follows:
spotter_ids = list(spotter.keys())

In [ ]:
# pick one of the spotters
sid = spotter_ids[0]
# keep ones with wave data
only_waves = spotter[sid]['energy_density'].notnull()
spotter[sid][only_waves]

In [ ]:
# pick a time
time_index = 36

# pull arrays from the buoys
f  = spotter[sid][only_waves]['frequency'][time_index]
Ef = spotter[sid][only_waves]['energy_density'][time_index]
a1 = spotter[sid][only_waves]['a1'][time_index]
b1 = spotter[sid][only_waves]['b1'][time_index]
a2 = spotter[sid][only_waves]['a2'][time_index]
b2 = spotter[sid][only_waves]['b2'][time_index]

# first and second moments of 1D energy spectrum
m0 = np.trapz(Ef, f)
m1 = np.trapz(Ef * f, f)

integrated_a1 = np.trapz(a1 * Ef, f) / m0
integrated_b1 = np.trapz(b1 * Ef, f) / m0

# Directional arrays (f)
D1 = (-180./np.pi)*np.arctan2(b1,a1)+90
D2 = (-180./np.pi)*0.5*np.arctan2(b2,a2)+90

# Spread1 seems to be the standard spreading parameter (O'Reilly et al., 1996; Jim Thomson's code...)
# Spread2 is an alternative proposed by O'Reilly et al. and appears in Jim Thomson's code
# Spread_alt is in Jim Thomson's code...not sure where it comes from
spread1 = (180./np.pi)*np.sqrt(2.*(1.-np.sqrt( a1**2 + b1**2)))
spread2 = (180./np.pi)*np.sqrt(np.abs( 0.5 - 0.5 * ( a2*np.cos(2.*D2) + b2*np.sin(2.*D2) )  ))
spread_alt = (180./np.pi)*np.sqrt(np.abs( 0.5 - 0.5 * ( a2**2 + b2**2 )  ))

# peak parameters
iEfmax = np.argmax(Ef)
Tp = 1./f[iEfmax]
Dp = D1[iEfmax]
S1p = spread1[iEfmax]
S2p = spread2[iEfmax]
Sap = spread_alt[iEfmax]

# standard bulk parameters
Hs = 4 * np.sqrt(m0)
# code from Isabel
Dm = np.arctan2(integrated_b1, integrated_a1) * 180 / np.pi
if(Dm<0.): Dm=Dm+360.
Tm = m0 / m1

# use model to make a 2dspec for spotter to compare round-tripping bulk stats
directions = np.linspace(5,355,36)
directional_bin_width = 10.
spotter_spectrum_mem2 = (
        estimate_directional_distribution(
            a1,
            b1,
            a2,
            b2,
            direction=directions,
            method="mem2",
        ).T
        * Ef
)
# this method of construction results in:
faxis = 1
daxis = 0
print('Check shape of spectrum: ',np.shape(spotter_spectrum_mem2))
print('freq. axis = ',faxis,' direc. axis = ',daxis,', right?')

spotter_spec1d = calc_spec1d( spotter_spectrum_mem2, directional_bin_width, daxis=daxis )

In [ ]:
eps = np.finfo(float).eps
print(eps)
D = np.divide( spotter_spectrum_mem2, spotter_spec1d+eps )
np.shape(D)

In [ ]:
print(np.sum(D))
print(np.sum(spotter_spectrum_mem2))
print(np.sum(spotter_spec1d))

In [ ]:
# calc some bulk statistics based on the 2d spec
Hs_spec2d = calc_Hs_2d( spotter_spectrum_mem2, f, directional_bin_width, faxis=faxis, daxis=daxis )
TM01_spec2d = calc_TM01_2d( spotter_spectrum_mem2, f, directional_bin_width, faxis=faxis, daxis=daxis )
TM02_spec2d = calc_TM02_2d( spotter_spectrum_mem2, f, directional_bin_width, faxis=faxis, daxis=daxis )

H0_spec1d = 4.*np.sqrt( calc_m0(  spotter_spec1d, f) )
FSPR = calc_FSPR( spotter_spec1d, f, TM02_spec2d )

# Hs is very close
# Tp is exactly right
# Tm is very close
# Dm is off
# Dp is off by 180
# S1P is exactly right

# Don't have estimates of S1P or Saltp in spotter database
# Don't know how to calculate mean direction or mean directional spread
# Unsure of directional conventions
print('Hs: ', Hs, spotter[sid][only_waves]['significant_height'][time_index], Hs_spec2d, H0_spec1d)
print('Tp: ', Tp, spotter[sid][only_waves]['peak_period'][time_index])
print('Tm: ', Tm, spotter[sid][only_waves]['mean_period'][time_index])
print('TM01_2dspec: ', TM01_spec2d, 'TM02_2dspec: ',TM02_spec2d )
print('Dm: ', Dm, spotter[sid][only_waves]['mean_direction'][time_index])
print('Dp: ', Dp, spotter[sid][only_waves]['peak_direction'][time_index])
print('S1p:' , S1p,spotter[sid][only_waves]['peak_directional_spread'][time_index])
print('S2p, Saltp: ', S2p, Sap)
print('FSPR:', FSPR )

In [ ]:
omega=2*np.pi*f
print(omega)
i = 0 + 1.j
print(i)
print(np.abs( np.exp(i*omega*10) ))
